<a href="https://colab.research.google.com/github/placerda/region-growing/blob/master/segmentation_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook to compare the automatic segmentation against ground truth.

#Preparation

In [0]:
# import and install libraries

!pip install pycuda pydicom
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

# if necessary include custom python modules use the except below
# import sys 
# sys.path.append('folder_name')

import numpy as np
import pydicom
import pycuda.autoinit 
from pycuda import gpuarray
from pycuda.compiler import SourceModule
import cv2
import json
%matplotlib inline
from matplotlib import pyplot as plt
import os
import pandas as pd

In [0]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#Utility Functions

In [0]:
# Flood fill (will be used to fill manually segmented area)
ff_kernel = SourceModule("""
#define _HEIGHT 512
#define _WIDTH 512

__device__ int get_index(int x, int y){
    return (x + y * _WIDTH);
}

__global__ void flood_fill(int *d_gt_borda, int *d_gt_area, float *d_debug){
    
    int x = threadIdx.x + blockIdx.x * blockDim.x; 
    int y = threadIdx.y + blockIdx.y * blockDim.y;

    int idx = get_index(x, y);

    if ((x < _WIDTH) && (y < _HEIGHT)){

        // left to right
        if (x == 0){
             for (int i = x; i < _WIDTH; i++){
                 int idx2 = get_index(i, y);
                 if (d_gt_borda[idx2] == 0)
                    d_gt_area[idx2] = 0;
                 else
                    break;
            }
        }

        // right to left
        if (x == _WIDTH-1){
             for (int i = x; i > 0; i--){
                 int idx2 = get_index(i, y);
                 if (d_gt_borda[idx2] == 0)
                    d_gt_area[idx2] = 0;
                 else
                    break;
            }
        }

        // top - down
        if (y == 0){
             for (int i = y; i < _HEIGHT; i++){
                 int idx2 = get_index(x, i);
                 if (d_gt_borda[idx2] == 0)
                    d_gt_area[idx2] = 0;
                 else
                    break;
            }
        }

        // bottom - up
        if (y == _HEIGHT-1){
             for (int i = y; i > 0; i--){
                 int idx2 = get_index(x, i);
                 if (d_gt_borda[idx2] == 0)
                    d_gt_area[idx2] = 0;
                 else
                    break;
            }
        }        
    }   
}
""")    

# Find contours (will be used to find contours in segmented area)
kernel_fc = SourceModule("""
# define _SIZE 512

__device__ int get_index(int x, int y){
    return x + y * _SIZE;
}

__device__ int is_extreme(int x, int y){
    if ((x == 0)
      ||(y == 0)
      ||(x == _SIZE - 1)
      ||(y == _SIZE - 1))
        return true;
    else
        return false;
}

__global__ void find_contours(int *d_segmented_area, int *d_segmented_contour){
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int idx = get_index(x, y);
    if ((x < _SIZE) && (y < _SIZE)){
        if (d_segmented_area[idx] == 1){
            // verifica se tem vizinho com valor == 0, caso afirmativo inclui (é borda)
            if (is_extreme(x,y))
                d_segmented_contour[idx] = 1;
            else if (d_segmented_area[get_index(x-1, y)] == 0)
                d_segmented_contour[idx] = 1;
            else if (d_segmented_area[get_index(x+1, y)] == 0)
                d_segmented_contour[idx] = 1;
            else if (d_segmented_area[get_index(x, y+1)] == 0)
                d_segmented_contour[idx] = 1;
            else if (d_segmented_area[get_index(x, y-1)] == 0)
                d_segmented_contour[idx] = 1;
        }
    }
}

""")

# Generates image with both segmented and manual contours for comparison
def generate_image(dicom_image, gt_contour, segmented_contour, slice_string,
                   y1, y2, x1, x2):

    # load dicom
    ds = pydicom.dcmread(dicom_image)
    b = ds.RescaleIntercept
    m = ds.RescaleSlope
    slice = m * ds.pixel_array + b
    slice = np.int32(slice)

    # quantization parameters
    WINDOW_LENGHT=40
    WINDOW_WIDTH=400
    MIN_HU=-1024
    MAX_HU=1024

    # quantization
    min_value = WINDOW_LENGHT - (WINDOW_WIDTH // 2)
    if min_value < MIN_HU: min_value = MIN_HU
    max_value = WINDOW_LENGHT + (WINDOW_WIDTH // 2)
    if max_value > MAX_HU: max_value = MAX_HU
    quantized = slice.copy()
    quantized = np.clip(quantized, min_value, max_value)
    for cell in np.nditer(quantized, op_flags=['readwrite']):
        cell[...] = ((cell - min_value) * 255) // (max_value - min_value)
    gray = quantized.astype('uint8')
    image_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)

    # add segmented contour in green
    overlay = image_rgb.copy()
    overlay[segmented_contour>0] = (0,255,0)

    # add manual annotation in red
    overlay[gt_contour>0] = (255,0,0)

    # get only ROI
    overlay = overlay[y1:y2, x1:x2]

    # add slice's number
    cv2.putText(overlay, "{}".format(slice_string), ((x2-x1-60), 30), 
                cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 
                lineType=cv2.LINE_AA)
    return overlay

# calculates if each pixel in segmented area is TP, FP, TN or FN
kernel = SourceModule("""
#define _SIZE 512

// 1: TP, 2: FP; 3: TN; 4: FN
__global__ void calculates_segmentation_results(int *d_gt_area, int *d_segmented_area, int *d_segmentation_result){
    
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int idx = x + y * _SIZE;
    if ((x < _SIZE) && (y < _SIZE)){
        if ((d_gt_area[idx]==1) && (d_segmented_area[idx]==1)){
            d_segmentation_result[idx] = 1; // TP
        } else if ((d_gt_area[idx]==0) && (d_segmented_area[idx]==1)){
            d_segmentation_result[idx] = 2; // FP
        } else if ((d_gt_area[idx]==0) && (d_segmented_area[idx]==0)){
            d_segmentation_result[idx] = 3; // TN
        } else if ((d_gt_area[idx]==1) && (d_segmented_area[idx]==0)){
            d_segmentation_result[idx] = 4; // FN
        }
    } 



}

""")

# Main Program

In [0]:
# dataset input and output folders
DICOM_FOLDER   = '/content/gdrive/My Drive/data/ratos/dataset'
GROUND_FOLDER  = '/content/gdrive/My Drive/data/ratos/groundtruth'
SEGMENTATION_FOLDER = '/content/gdrive/My Drive/data/ratos/results'
COMPARISON_FOLDER = '/content/gdrive/My Drive/data/ratos/comparison'

# filter scans to be compared
SCANS = ['Imacx01_Animal1', 'Imacx01Animal2', 'Imacx2Animal2', 
         'Imacx2Animal3', 'Imacx2Animal5']

WIDTH = 512
HEIGHT = 512

#  Dataframe with metrics
metrics_df = pd.DataFrame(columns=['Scan', 'Slice', 'TP', 'FP', 'TN', 'FN', 
                                   'GT Area', 'ACC', 'PDP', 'PDN', 'ROI-INDEX',
                                   'SEN', 'SPE', 'EFI', 'Y-INDEX', 'DICE'])

for scan in SCANS:
    print('\nprocessing scan: {}'.format(scan))

    # process each ground truth annotation file
    ground_folder = '{}/{}'.format(GROUND_FOLDER, scan)
    files = os.listdir(ground_folder)
    files.sort()
    
    for file in files:

        # get slice's number from the filename (ex: 122 from IM-0001-0122.json)
        temp_list = file.split('-')
        temp_str = temp_list[len(temp_list)-1]
        slice_string = temp_str[:temp_str.find('.json')]
        slice_number = int(slice_string)

        print('{} '.format(slice_string), end='' )

        # get file paths
        ground_file = '{}/{}'.format(ground_folder, file)
        segmentation_file = '{}/{}/segmented_data.npy'.format(SEGMENTATION_FOLDER, scan)
        dicom_file = '{}/{}/IM-0001-{}.dcm'.format(DICOM_FOLDER, scan, slice_string)

        # initializes masks
        gt_contour = np.zeros((HEIGHT,WIDTH), dtype='int32')
        segmented_contour = np.zeros((HEIGHT,WIDTH), dtype='int32')
        gt_area = np.zeros((HEIGHT,WIDTH), dtype='int32')
        segmented_area = np.zeros((HEIGHT,WIDTH), dtype='int32')

        # first step - process ground truth

        with open(ground_file) as json_file:
            gt_data = []
            gt_data = json.load(json_file)

            # gets ground truth contour
            for contour in gt_data:
                i = 0
                # round all items in the points list
                round_list = lambda lista : [int(round(item, 0)) for item in lista]                
                points = [round_list(p) for p in contour]
                last_point = []
                for point in points:
                    if i == 0:
                        # add countour points
                        gt_contour[point[1], point[0]] = 1
                        last_point = point
                    else:
                        cv2.line(gt_contour, (last_point[0],last_point[1]), 
                                 (point[0], point[1]), 1, 1)
                        last_point = point
                    i+=1

            # gets ground truth area
            d_gt_contour = gpuarray.to_gpu(gt_contour)
            d_gt_area = gpuarray.to_gpu(np.ones_like(gt_contour, dtype='int32'))
            h_debug = np.zeros((1), dtype='float32')    
            d_debug = gpuarray.to_gpu(h_debug)
            flood_fill = ff_kernel.get_function("flood_fill")
            flood_fill(d_gt_contour, d_gt_area, d_debug, 
                       grid=(16,16,1), block=(32,32,1))
            gt_area = d_gt_area.get()

        # second step - process segmented volume
        
        # gets segmented area
        segmented_data = np.load(segmentation_file)
        segmented_area = segmented_data[slice_number-1]
        d_segmented_area = gpuarray.to_gpu(segmented_area.astype('int32'))
         
        # get segmentation contour
        d_segmented_contour = gpuarray.to_gpu(segmented_contour)
        find_contours = kernel_fc.get_function('find_contours')
        find_contours(d_segmented_area, d_segmented_contour, 
                      grid=(16, 16, 1), block=(32, 32, 1))
        segmented_contour = d_segmented_contour.get()

        # third step - generates comparison image
        
        comparison_image = generate_image(dicom_file, gt_contour, 
                                      segmented_contour, slice_string,
                                      100, 300, 120, 400)

        # save image
        comparison_folder = '{}/{}'.format(COMPARISON_FOLDER, scan)
        os.makedirs(comparison_folder, exist_ok=True)

        comparison_file = '{}/comparison-{}.png'.format(comparison_folder, 
                                                        slice_number)
        cv2.imwrite(comparison_file, 
                    cv2.cvtColor(comparison_image, cv2.COLOR_RGB2BGR))
        # shows image
        # plt.imshow(comparison_image)
        # plt.show()

        # fourth step - calculate metrics

        segmentation_result = np.zeros_like(segmented_area, dtype='int32')
        d_segmentation_result = gpuarray.to_gpu(segmentation_result)

        calculates_segmentation_results = kernel.get_function('calculates_segmentation_results')
        calculates_segmentation_results(d_gt_area, d_segmented_area, 
                                        d_segmentation_result, 
                                        grid=(16,16,1), block=(32,32,1))
        segmentation_result = d_segmentation_result.get()

        a_row = {}
        a_row['Scan'] = scan
        a_row['Slice'] = slice_string

        tp = np.count_nonzero(segmentation_result == 1)
        fp = np.count_nonzero(segmentation_result == 2)
        tn = np.count_nonzero(segmentation_result == 3)
        fn = np.count_nonzero(segmentation_result == 4)
        acc = (tp + tn) / (tp + fp + tn + fn)
        pdp = tp / (tp+fp) # precision
        pdn = tn / (tn+fn)
        roi_index = 100 * (1 - (acc * pdn * pdp))
        sen = tp / (tp+fn) # recall
        spe = tn / (tn+fp) # specifity
        efi = (0.5) * (sen + spe)
        y_index = (sen + spe -1)
        dice = (2*tp)/((2*tp) + fp + fn)

        a_row['TP'] = tp
        a_row['FP'] = fp
        a_row['TN'] = tn
        a_row['FN'] = fn
        a_row['GT Area'] = np.sum(gt_area)
        a_row['ACC'] = acc
        a_row['PDP'] = pdp
        a_row['PDN'] = pdn
        a_row['ROI-INDEX'] = roi_index
        a_row['SEN'] = sen
        a_row['SPE'] = spe
        a_row['EFI'] = efi
        a_row['Y-INDEX'] = y_index
        a_row['DICE'] = dice

        metrics_df = metrics_df.append(a_row , ignore_index=True)

metrics_df